In [43]:
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from tensorflow import keras 
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.optimizers import Adam 

from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Generic Model Parameters

In [44]:
class ModelParameters:
    def __init__(self):
        self.Special_Layer_Type = layers.SimpleRNN
        self.Special_Layer_Num = 1
        self.Special_Layer_Units = 4
        self.Special_Layer_Activation = "tanh"
        self.Dense_Layer_Num = 1
        self.Dense_Layer_Units = 4
        self.Dense_Layer_Activation = None 
        self.Output_Layer_Units = 1 
        self.Output_Layer_Activation = "linear"
        self.Epochs = 10
        self.Window_Length = 5
        self.Input_Scaler = StandardScaler()
        self.Output_Scaler = StandardScaler()
        self.Training_Size = .70
        self.Validation_Size = .20
        self.ModelName = "HatdogModel"

# Generic Network Workflow

In [45]:
class NeuralNetworkWorkflow:
    def __init__(self, ParameterClass, DataFileName, DataColumnName):
        self.pc : ModelParameters = ParameterClass
        self.raw_df = pd.read_csv(DataFileName)
        self.target_column = DataColumnName

    def preprocess_data(self):
        # Copy Loaded DF 
        self.df = self.raw_df.copy()

        # Hold Values into a temporary numpy array
        t_arr = self.df[self.target_column].to_numpy()
        X, y = [], []
        
        # Iterate through elements of the numpy array until end - window_length
        for i in range(len(t_arr) - self.pc.Window_Length):
            t_row = []
            for j in t_arr[i : i + self.pc.Window_Length]:
                t_row.append([j])
            X.append(t_row)
            y.append(t_arr[i + self.pc.Window_Length])

        # Set curreny X and Y as Numpy Arrays (For Scaling)
        X, y  = np.array(X), np.array(y)

        # Create Scalers
        in_scaler, out_scaler = self.pc.Input_Scaler, self.pc.Output_Scaler

        # Create Copies
        X_copy, y_copy = X.copy(), y.copy()

        # Reshape X to Fit and Transform to Input Scaler
        X = X.reshape(X.shape[0], X.shape[1])
        X = in_scaler.fit_transform(X)
        X = X.reshape(X_copy.shape[0], X_copy.shape[1], X_copy.shape[2])
        print(X.shape)

        # Reshape Y to Fit and Transform to Output Scaler
        y = y.reshape(-1, 1)
        y = out_scaler.fit_transform(y)
        y = y.flatten()

        # Split into Training, Testing and Validation Parts
        train_part = int(self.pc.Training_Size * len(y))
        valid_part = int(self.pc.Validation_Size * len(y))

        # Assignments
        self.X_train, self.y_train = X[:train_part], y[:train_part]
        self.X_val, self.y_val = X[train_part:valid_part], y[train_part:valid_part]
        self.X_test, self.y_test = X[valid_part:], y[valid_part:]
        print(np.array(X))

        # Preserve for Testing
        self.y_test_copy = self.y_test.copy()

    def build_model(self):
        self.X_train = np.array(self.X_train)
        print(self.X_train.shape)
        model = Sequential()
        
        # Input Layer
        model.add(layers.InputLayer(self.pc.Window_Length, 1, ))

        # Special Layers
        if self.pc.Special_Layer_Num > 1:
            for _ in range(self.pc.Special_Layer_Num):
                model.add(self.pc.Special_Layer_Type(self.pc.Special_Layer_Units, return_sequences=True, activation=self.pc.Special_Layer_Activation))
        
        # Final Special Layer
        model.add(self.pc.Special_Layer_Type(self.pc.Special_Layer_Units, activation=self.pc.Special_Layer_Activation))

        # Dense Layers
        for _ in range(self.pc.Dense_Layer_Num):
            model.add(layers.Dense(self.pc.Dense_Layer_Units, activation=self.pc.Dense_Layer_Activation))

        # Output Layer
        model.add(layers.Dense(self.pc.Output_Layer_Units, activation=self.pc.Output_Layer_Activation))
        
        # Filepath for Checkpoint
        filepath = self.pc.ModelName + "/"
        cp = ModelCheckpoint(filepath, save_best_only=True)

        # Compile Model
        model.compile(
            loss = MeanSquaredError(),
            optimizer = Adam(learning_rate=0.0001),
            metrics = [RootMeanSquaredError()]
        )

        # Fit Model (Saves History)
        history = model.fit(
            self.X_train,
            self.y_train,
            validation_data=(self.X_val, self.y_val),
            epochs = self.pc.Epochs,
            callbacks = [cp],
            verbose = 1
        )
        
        # Load Model
        model = load_model("LSTMTestModel/")

        self.trained_model = model 

    def show_historical_plots(self):
        pass 

    def show_evaluation_results(self):
        # Non Transformed
        y_test = self.y_test_copy.copy()
        input_arr = self.X_test.reshape(self.X_test.shape[0], self.X_test.shape[1])
        test_predictions = model.predict(input_arr).flatten()
        u = 1
        l = 100
        plt.figure(figsize=(15, 5))
        plt.subplot(121)
        plt.plot(test_predictions[u:l], label="Predictions", color="green")
        plt.legend()
        plt.subplot(122)
        plt.plot(y_test[u:l], label="Actual Test", color="orange")
        plt.legend()

        # Inverse Transformed
        y_test = self.y_test_copy.copy()
        test_predictions = model.predict(input_arr).flatten()
        y_test = y_test.reshape(-1, 1)
        y_test = out_scaler.inverse_transform(y_test)
        test_predictions = test_predictions.reshape(-1, 1)
        test_predictions = out_scaler.inverse_transform(test_predictions)
        plt.figure(figsize=(15, 5))
        plt.subplot(123)
        plt.plot(test_predictions[u:l], label="[O] Predictions", color="blue")
        plt.legend()
        plt.subplot(124)
        plt.plot(y_test[u:l], label="[O] Actual Test", color="red")
        plt.legend()
        plt.show()

    def default_run(self):
        self.preprocess_data()
        self.build_model()
        self.show_evaluation_results()

### Precipitation Predictor
Applies Standard Scaler

In [46]:
class PrecipitationPredictorParameters(ModelParameters):
    def __init__(self):
        super().__init__()
        self.Special_Layer_Num = 3
        self.Special_Layer_Units = 64
        self.Dense_Layer_Num = 3
        self.Dense_Layer_Units = 64
        self.Output_Layer_Activation = None
        self.Epochs = 10
        self.Window_Length = 5


In [47]:
PPP = PrecipitationPredictorParameters()
New_Flow = NeuralNetworkWorkflow(PPP, "qc aggregated data.csv", "precip")

In [48]:
New_Flow.default_run()

(1303, 5, 1)
[[[-0.40352016]
  [-0.40785738]
  [-0.41049622]
  [-0.41074516]
  [-0.38751205]]

 [[-0.40661442]
  [-0.41094716]
  [-0.41018488]
  [-0.3865293 ]
  [-0.41202801]]

 [[-0.40970867]
  [-0.41063188]
  [-0.38577584]
  [-0.41105403]
  [-0.40931087]]

 ...

 [[-0.26124759]
  [-0.12321868]
  [ 4.41016554]
  [ 1.93121285]
  [ 5.65199728]]

 [[-0.12156408]
  [ 4.4707893 ]
  [ 1.95045616]
  [ 5.6551429 ]
  [ 4.5462473 ]]

 [[ 4.47908769]
  [ 1.97991698]
  [ 5.70409398]
  [ 4.54899693]
  [ 1.04911074]]]
(912, 5, 1)


ValueError: Input 0 of layer "simple_rnn_7" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (1, 5)